In [ ]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.utilities import WikipediaAPIWrapper

llm = ChatOpenAI(temperature=0.1)

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)
    
class StockMarketSymbolSearchToolWIKI(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    This tool uses Wikipedia to find the stock market symbol for a company.
    Provide the name of the company as a query, and the tool will attempt to retrieve its stock market symbol from Wikipedia articles.
    Perfect for quickly finding the ticker symbols of well-known companies through the comprehensive information available on Wikipedia.
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = WikipediaAPIWrapper()
        return ddg.run(query)

class SaveResultsTool(BaseTool):
    name = "SaveResultsTool"
    description = "Saves given text to a .txt file."

    def _run(self, text: str, filename: str) -> str:
        with open(filename, "w", encoding="utf-8") as file:
            file.write(text)
        return f"Results saved to {filename}"

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
    ],
)

prompt = "Research about the XZ backdoor"

agent.invoke(prompt)
